In [1]:
import psycopg2

# Winning numbers
white_balls = [6, 15, 18, 33, 49]
powerball = 7

# Normalize the numbers
normalized_white_balls = [num / 69.0 for num in white_balls]
normalized_powerball = powerball / 26.0
winning_vector = normalized_white_balls + [normalized_powerball]

# SQL query to find similar tickets
query = f"""
SELECT *, embedding <-> '{winning_vector}'::vector AS similarity
FROM powerball_ticket_sessions
ORDER BY similarity
LIMIT 5;
"""

def find_similar_tickets():
    try:
        # Connect to PostgreSQL
        conn = psycopg2.connect(
            dbname="rag_db",
            user="user",
            password="password",
            host="192.168.0.89",
            port=5432
        )
        cursor = conn.cursor()

        # Execute the query
        cursor.execute(query)
        results = cursor.fetchall()

        # Print the results
        print("Most similar tickets to the winning numbers:")
        for row in results:
            print(row)
    except Exception as e:
        print(f"Database error: {e}")
    finally:
        cursor.close()
        conn.close()

# Run the function
find_similar_tickets()


Most similar tickets to the winning numbers:
(131, 3, 15, 16, 34, 57, 9, 'classical', 243531, '[0.04347826,0.2173913,0.23188406,0.49275362,0.82608694,0.34615386]', datetime.datetime(2024, 12, 19, 6, 1, 12, 451066), 0.14933273495596114)
(178, 4, 13, 15, 37, 54, 3, 'classical', 243531, '[0.057971016,0.1884058,0.2173913,0.5362319,0.7826087,0.115384616]', datetime.datetime(2024, 12, 19, 6, 7, 43, 224241), 0.1893434895527659)
(128, 7, 10, 29, 38, 52, 8, 'classical', 243531, '[0.10144927,0.14492753,0.42028984,0.5507246,0.7536232,0.30769232]', datetime.datetime(2024, 12, 19, 6, 0, 46, 993169), 0.1987372732286307)
(199, 9, 14, 21, 23, 58, 8, 'classical', 243531, '[0.13043478,0.20289855,0.3043478,0.33333334,0.8405797,0.30769232]', datetime.datetime(2024, 12, 19, 6, 10, 47, 579577), 0.20853598791451253)
(87, 6, 22, 26, 32, 43, 4, 'classical', 243531, '[0.08695652,0.3188406,0.3768116,0.46376812,0.6231884,0.15384616]', datetime.datetime(2024, 12, 19, 5, 49, 47, 241914), 0.21170642014527247)
